# Combine parcellations

Here, two parcellations are combined into one. These parcellations are:
- base parcellation: whole-brain network functional parcellation from [Power et al., 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3222858/)
- meta-analytic parcellation: only regions involved in processing positive and negative prediction errors from [Fouragnan et al., 2018](https://pubmed.ncbi.nlm.nih.gov/29575249/)

There are three main strategies for combining individual ROIs:

![strategy](parcellation_combine_strategy.png)

In [1]:
import os

import pandas as pd

from os.path import join
from pathlib import Path
from dn_utils.path import path_derivatives, path_parcellations

In [2]:
path_root = os.environ.get('DECIDENET_PATH')
path_derivatives = join(path_root, 'data/main_fmri_study/derivatives')
path_parcellations = join(path_derivatives, 'parcellations')

df_roi_base = pd.read_csv(
    join(path_parcellations, 'Power_roi/Power_roi_table.csv'))
df_roi_meta = pd.read_csv(
    join(path_parcellations, 'meta_roi_4and5/meta_roi_table.csv'))

In [3]:
df_roi_meta.head()

,abbrev,name,hemisphere,x,y,z,cluster_size,ale_score,netName,approxRadius,radius(mm)
0,dMCC,Dorsomedial cingulate cortex,R,2,24,36,12712,0.051,perr_dec,14.477995,5
1,aINS,Anterior insula,R,32,24,-2,6120,0.062,perr_dec,11.347171,5
2,aINS,Anterior insula,L,-32,22,-4,4880,0.056,perr_dec,10.522293,5
3,PAL,Pallidum,R,12,8,4,3360,0.040,perr_dec,9.291451,4
4,PAL,Pallidum,L,-14,6,2,2520,0.029,perr_dec,8.441844,4


In [4]:
df_roi_new = df_roi_base.copy()
df_roi_new['origNetName'] = df_roi_new['netName']
df_roi_new['strategy'] = None
df_roi_new['origName'] = None

shared_columns = ['x', 'y', 'z', 'netName', 'radius(mm)']

for i, roi_meta in df_roi_meta.iterrows():

    roi_meta_name = roi_meta['abbrev'] + '_' + roi_meta['hemisphere']
    
    # Calculate distance between meta ROI and all ROIs from base atlas
    dist = df_roi_base.loc[:, ['x', 'y', 'z']] - roi_meta[['x', 'y', 'z']]
    dist = dist.pow(2).sum(axis=1).pow(0.5)

    # Grab closest region
    roi_base_idx = dist.argmin()
    roi_base = df_roi_base.loc[roi_base_idx]
    
    # Max radius of newly created ROI (non-overlapping)
    max_r = dist[roi_base_idx] - roi_base['radius(mm)']

    print(f'ROI: {roi_meta_name} ({roi_meta["netName"]})\n\t' + \
          f'• closest roi: {roi_base_idx} ({roi_base["netName"]}, r={roi_base["radius(mm)"]}mm)\n\t' + \
          f'• distance = {dist[roi_base_idx]:4.2f}mm, max_r={max_r:4.2f}mm')
    
    if max_r >= 5:
        # Create new ROI
        df_roi_new = df_roi_new.append(roi_meta[shared_columns], 
                                       ignore_index=True)
        df_roi_new.iloc[-1, df_roi_new.columns.get_loc('strategy')] = 'created'
        df_roi_new.iloc[-1, df_roi_new.columns.get_loc('origName')] = roi_meta_name
        
        print(f'\t✅ created new ROI: {roi_meta_name} ({roi_meta["netName"]})\n')
    
    elif max_r > 0:
        # Shift old ROI and rename network
        df_roi_new.loc[roi_base_idx, shared_columns] = roi_meta[shared_columns]
        df_roi_new.loc[roi_base_idx, 'strategy'] = 'shifted'
        df_roi_new.loc[roi_base_idx, 'origName'] = roi_meta_name
        
        before_xyz = roi_base[['x', 'y', 'z']].astype(str).str.cat(sep=', ')
        after_xyz = roi_meta[['x', 'y', 'z']].astype(str).str.cat(sep=', ')
        
        print(f'\t✅ shifted: ({before_xyz}) ⮕ ({after_xyz})')
        print(f'\t✅ relabelled: {roi_base["netName"]} ⮕ {roi_meta["netName"]}\n')
    
    else:
        # Just change network label
        df_roi_new.loc[roi_base_idx, 'netName'] = roi_meta['netName']
        df_roi_new.loc[roi_base_idx, 'strategy'] = 'relabelled'
        df_roi_new.loc[roi_base_idx, 'origName'] = roi_meta_name
        
        print(f'\t✅ relabelled: {roi_base["netName"]} ⮕ {roi_meta["netName"]}\n')

ROI: dMCC_R (perr_dec)
	• closest roi: 215 (salience, r=5mm)
	• distance = 3.32mm, max_r=-1.68mm
	✅ relabelled: salience ⮕ perr_dec

ROI: aINS_R (perr_dec)
	• closest roi: 208 (salience, r=5mm)
	• distance = 6.71mm, max_r=1.71mm
	✅ shifted: (36, 22, 3) ⮕ (32, 24, -2)
	✅ relabelled: salience ⮕ perr_dec

ROI: aINS_L (perr_dec)
	• closest roi: 207 (salience, r=5mm)
	• distance = 5.39mm, max_r=0.39mm
	✅ shifted: (-35, 20, 0) ⮕ (-32, 22, -4)
	✅ relabelled: salience ⮕ perr_dec

ROI: PAL_R (perr_dec)
	• closest roi: 232 (subcortical, r=5mm)
	• distance = 5.20mm, max_r=0.20mm
	✅ shifted: (15, 5, 7) ⮕ (12, 8, 4)
	✅ relabelled: subcortical ⮕ perr_dec

ROI: PAL_L (perr_dec)
	• closest roi: 227 (subcortical, r=5mm)
	• distance = 6.40mm, max_r=1.40mm
	✅ shifted: (-15, 4, 8) ⮕ (-14, 6, 2)
	✅ relabelled: subcortical ⮕ perr_dec

ROI: mFG1_R (perr_dec)
	• closest roi: 185 (fronto_parietal, r=5mm)
	• distance = 10.86mm, max_r=5.86mm
	✅ created new ROI: mFG1_R (perr_dec)

ROI: mFG2_R (perr_dec)
	• closes

In [5]:
df_roi_new.loc[df_roi_new['netName'] == 'perr_inc']

,x,y,z,netCode,netColor,netName,radius(mm),origNetName,strategy,origName
11,32,44,-10,-1.0,White,perr_inc,5,uncertain,shifted,vlOFC_R
89,-6,-56,14,5.0,Red,perr_inc,5,default_mode,shifted,dmPFC_L
108,-2,42,0,5.0,Red,perr_inc,5,default_mode,shifted,vmPFC_L
112,-2,46,20,5.0,Red,perr_inc,5,default_mode,shifted,mPFC_L
132,0,-36,26,6.0,Gray,perr_inc,5,memory,shifted,PCC_L
270,-12,8,-4,NaN,NaN,perr_inc,4,NaN,created,vSTR_L
271,8,8,-2,NaN,NaN,perr_inc,4,NaN,created,vSTR_R


In [6]:
df_roi_new.loc[df_roi_new['netName'] == 'perr_dec']

,x,y,z,netCode,netColor,netName,radius(mm),origNetName,strategy,origName
118,60,-28,-6,5.0,Red,perr_dec,5,default_mode,shifted,mTG_R
177,-28,12,60,8.0,Yellow,perr_dec,5,fronto_parietal,shifted,mFG_L
191,40,-48,42,8.0,Yellow,perr_dec,5,fronto_parietal,shifted,IPL_R
192,32,14,56,8.0,Yellow,perr_dec,5,fronto_parietal,relabelled,mFG2_R
200,-42,25,30,8.0,Yellow,perr_dec,5,fronto_parietal,relabelled,dlPFC_L
205,40,34,30,9.0,Black,perr_dec,5,salience,shifted,dlPFC_R
207,-32,22,-4,9.0,Black,perr_dec,5,salience,shifted,aINS_L
208,32,24,-2,9.0,Black,perr_dec,5,salience,shifted,aINS_R
215,5,23,37,9.0,Black,perr_dec,5,salience,relabelled,dMCC_R
223,-6,-26,8,10.0,Brown,perr_dec,5,subcortical,shifted,THA2_L


In [13]:
path_out = join(path_parcellations, 'combined_roi_4and5')
Path(path_out).mkdir(exist_ok=True)

df_roi_new = df_roi_new.sort_values(by='netName', axis=0, ascending=False)
df_roi_new.to_csv(join(path_out, 'combined_roi_4and5_table.csv'), index=False)

# For thesis
df_roi_new.loc[~df_roi_new['strategy'].isna()].to_csv(
    join(path_out, "combined_roi_4and5_new_rois_table.csv"), index=False)

In [14]:
df_roi_new

,x,y,z,netCode,netColor,netName,radius(mm),origNetName,strategy,origName
160,42,-66,-8,7.0,Blue,visual,5,visual,None,None
168,37,-84,13,7.0,Blue,visual,5,visual,None,None
167,-40,-88,-6,7.0,Blue,visual,5,visual,None,None
152,43,-78,-12,7.0,Blue,visual,5,visual,None,None
153,-47,-76,-10,7.0,Blue,visual,5,visual,None,None
...,...,...,...,...,...,...,...,...,...,...
69,-55,-9,12,4.0,Pink,auditory,5,auditory,None,None
72,-30,-27,12,4.0,Pink,auditory,5,auditory,None,None
71,59,-17,29,4.0,Pink,auditory,5,auditory,None,None
66,43,-23,20,4.0,Pink,auditory,5,auditory,None,None
